In [1]:
import os
import cv2
import random
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import itertools

from tqdm import tqdm
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix


warnings.filterwarnings('ignore')
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

## 导入数据
-  label 文件一共有 79572 行
- 各种维度混合在一起
- 我们的目的是切分开各种维度, 进行训练和模拟
- 那么首先取出个标签的图片进行聚类

In [2]:
df_train = pd.read_csv('./Annotations/label.csv', header=None)
df_train.columns = ['image_id', 'class', 'label']
df_train.head()

,image_id,class,label
0,Images/collar_design_labels/4d8a38b29930a403e5...,collar_design_labels,nnynn
1,Images/collar_design_labels/bd0981f231180d2b00...,collar_design_labels,nynnn
2,Images/collar_design_labels/26937e1724feadfe39...,collar_design_labels,ynnnn
3,Images/collar_design_labels/cf4140ec542887270f...,collar_design_labels,nynnn
4,Images/collar_design_labels/50644b2b9de045f2d1...,collar_design_labels,nynnn


In [3]:
classes = ['collar_design_labels', 'neckline_design_labels', 'skirt_length_labels',
          'sleeve_length_labels', 'neck_design_labels', 'coat_length_labels', 'lapel_design_labels',
          'pant_length_labels']

In [4]:
cur_class = classes[6]
df_load = df_train[(df_train['class'] == cur_class)].copy()
df_load.reset_index(inplace=True)
del df_load['index']

print('{0}: {1}'.format(cur_class, len(df_load)))
df_load.head()

lapel_design_labels: 7034


,image_id,class,label
0,Images/lapel_design_labels/6cd79483ca0040a67fe...,lapel_design_labels,nnnny
1,Images/lapel_design_labels/b169d0a8a9dd0301ff3...,lapel_design_labels,nynnn
2,Images/lapel_design_labels/2dd8c8d311247ca384a...,lapel_design_labels,nynnn
3,Images/lapel_design_labels/2da3f9e7243ab23014a...,lapel_design_labels,nnnny
4,Images/lapel_design_labels/47094db231099dc959c...,lapel_design_labels,nnynn


In [5]:
n = len(df_load)
n_class = len(df_load['label'][0])
width = 299 # 定义图片大小

X = np.zeros((n, width, width, 3), dtype=np.uint8)
y = np.zeros((n, n_class), dtype=np.uint8)

In [6]:
n_class

5

In [7]:
print('./{0}'.format(df_load['image_id'][0]))

./Images/lapel_design_labels/6cd79483ca0040a67fe70f3f67882955.jpg


In [8]:
for i in tqdm(range(n)):
    tmp_label = df_load['label'][i]
    if len(tmp_label) > n_class:
        print(df_load['image_id'][i])
    X[i] = cv2.resize(cv2.imread('./{0}'.format(df_load['image_id'][i])), (width, width))
    y[i][tmp_label.find('y')] = 1

100%|██████████| 7034/7034 [00:47<00:00, 148.15it/s]


In [10]:
X.shape, y.shape

((7034, 299, 299, 3), (7034, 5))

In [11]:
#plt.figure(figsize=(12, 7))
#for i in range(8):
#    random_index = random.randint(0, n-1)
#    plt.subplot(2, 4, i+1)
#    plt.imshow(X[random_index][:,:,::-1])
#    plt.title(y[random_index])

## 提取特征

In [9]:
from keras.layers import *
from keras.models import *
from keras.callbacks import *
from keras.optimizers import *
from keras.applications import *
from keras.regularizers import *
from keras.layers import *
from keras.models import *
from keras.callbacks import *
from keras.optimizers import *
from keras.applications import *
from keras.regularizers import *
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau
from keras.optimizers import RMSprop
import keras

Using TensorFlow backend.


In [10]:
cnn_model = InceptionV3(include_top=False, input_shape=(width, width, 3), weights='imagenet')

In [11]:
inputs = Input((width, width, 3))

x = inputs
x = Lambda(preprocess_input, name='preprocessing')(x)
x = cnn_model(x)
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
x = Dense(n_class, activation='softmax', name='softmax')(x)

model = Model(inputs, x)

In [12]:
prefix_cls = cur_class.split('_')[0]
prefix_cls

'lapel'

## 划分训练/测试集

In [16]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.12, random_state=42)
X_train.shape, y_train.shape

((6189, 299, 299, 3), (6189, 5))

In [17]:
# Compile the model
adam = Adam(lr=0.001) 

model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])
# Set a learning rate annealer
# learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc',
#                                             patience=3,
#                                             verbose=1,
#                                             factor=0.1,
#                                             min_lr=0.00001)
epochs = 16
batch_size = 32

## 数据增强

In [18]:
datagen = ImageDataGenerator(
        featurewise_center = False, # set input mean to 0 over the dataset
        samplewise_center = False, # set each sample mean to 0
        featurewise_std_normalization = False, # divide inputs by std of the dataset
        samplewise_std_normalization = False, # divide each input by its std
        zca_whitening = False, # apply ZCA whitening
        rotation_range = 10, # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.05, # randomly zoom image
        width_shift_range = 0.08, # randomly shift images horizontally (fraction of total width)
        height_shift_range = 0.08, # randomly shift images vertivally (fraction of total heigth)
        horizontal_flip = True, # randomly flip images
        vertical_flip = False,
        shear_range = 0.08,
        fill_mode = 'constant',
        cval = 0)

datagen.fit(X)


- 少量旋转
- 少量偏移
- 水平翻转
- 垂直翻转

In [19]:
prefix_cls = cur_class.split('_')[0]

checkpointer = ModelCheckpoint(filepath='./models/{0}.best0420_InceptionV3_alldata.h5'.format(prefix_cls), verbose=1, 
                               save_best_only=True, save_weights_only=True, mode='val_acc')

try:
    # Fit the model
    history = model.fit_generator(datagen.flow(X, y, batch_size=batch_size),
                                  epochs=epochs, validation_data=(X_valid, y_valid),
                                  verbose=2, steps_per_epoch=X.shape[0] // batch_size,
                                  callbacks=[EarlyStopping(patience=5), checkpointer])
except KeyboardInterrupt:
    print('KeyboardInterrupt')

Epoch 1/16
 - 139s - loss: 1.5088 - acc: 0.3350 - val_loss: 4.6275 - val_acc: 0.3432

Epoch 00001: val_loss improved from inf to 4.62750, saving model to ./models/lapel.best0420_InceptionV3_alldata.h5
Epoch 2/16
 - 135s - loss: 1.0369 - acc: 0.5824 - val_loss: 3.0922 - val_acc: 0.3858

Epoch 00002: val_loss improved from 4.62750 to 3.09218, saving model to ./models/lapel.best0420_InceptionV3_alldata.h5
Epoch 3/16
 - 135s - loss: 0.7595 - acc: 0.7118 - val_loss: 0.8110 - val_acc: 0.6970

Epoch 00003: val_loss improved from 3.09218 to 0.81098, saving model to ./models/lapel.best0420_InceptionV3_alldata.h5
Epoch 4/16
 - 136s - loss: 0.6293 - acc: 0.7752 - val_loss: 0.6422 - val_acc: 0.7680

Epoch 00004: val_loss improved from 0.81098 to 0.64224, saving model to ./models/lapel.best0420_InceptionV3_alldata.h5
Epoch 5/16
 - 135s - loss: 0.5568 - acc: 0.7958 - val_loss: 0.4725 - val_acc: 0.8355

Epoch 00005: val_loss improved from 0.64224 to 0.47250, saving model to ./models/lapel.best0420_In

In [20]:
cnn_model = InceptionV3(include_top=False, input_shape=(width, width, 3), weights='imagenet')

In [21]:
inputs = Input((width, width, 3))

x = inputs
x = Lambda(preprocess_input, name='preprocessing')(x)
x = cnn_model(x)
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
x = Dense(n_class, activation='softmax', name='softmax')(x)

model = Model(inputs, x)

In [22]:
model.load_weights('./models/{}.best0420_InceptionV3_alldata.h5'.format(prefix_cls)) #加载以前训练好的模型，继续测试

In [ ]:
# Compile the model
adam = Adam(lr=0.0001) 

model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])
# Set a learning rate annealer
# learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc',
#                                             patience=3,
#                                             verbose=1,
#                                             factor=0.1,
#                                             min_lr=0.00001)
epochs = 16
batch_size = 32

In [ ]:
prefix_cls = cur_class.split('_')[0]

checkpointer = ModelCheckpoint(filepath='./models/{0}.best0420_InceptionV3_alldata.h5'.format(prefix_cls), verbose=1, 
                               save_best_only=True, save_weights_only=True, mode='val_acc')

try:
    # Fit the model
    history = model.fit_generator(datagen.flow(X, y, batch_size=batch_size),
                                  epochs=epochs, validation_data=(X_valid, y_valid),
                                  verbose=2, steps_per_epoch=X.shape[0] // batch_size,
                                  callbacks=[EarlyStopping(patience=5), checkpointer])
except KeyboardInterrupt:
    print('KeyboardInterrupt')

Epoch 1/16
 - 141s - loss: 0.3378 - acc: 0.8796 - val_loss: 0.2183 - val_acc: 0.9325

Epoch 00001: val_loss improved from inf to 0.21827, saving model to ./models/lapel.best0420_InceptionV3_alldata.h5
Epoch 2/16
 - 138s - loss: 0.2676 - acc: 0.9110 - val_loss: 0.1852 - val_acc: 0.9408

Epoch 00002: val_loss improved from 0.21827 to 0.18525, saving model to ./models/lapel.best0420_InceptionV3_alldata.h5
Epoch 3/16
 - 137s - loss: 0.2250 - acc: 0.9198 - val_loss: 0.1675 - val_acc: 0.9420

Epoch 00003: val_loss improved from 0.18525 to 0.16752, saving model to ./models/lapel.best0420_InceptionV3_alldata.h5
Epoch 4/16
 - 137s - loss: 0.1953 - acc: 0.9364 - val_loss: 0.1194 - val_acc: 0.9645

Epoch 00004: val_loss improved from 0.16752 to 0.11938, saving model to ./models/lapel.best0420_InceptionV3_alldata.h5
Epoch 5/16
 - 135s - loss: 0.1789 - acc: 0.9381 - val_loss: 0.1018 - val_acc: 0.9645

Epoch 00005: val_loss improved from 0.11938 to 0.10179, saving model to ./models/lapel.best0420_In

## 加载模型

In [13]:
df_test = pd.read_csv('./z_rank/Tests/question.csv', header=None)
df_test.columns = ['image_id', 'class', 'x']
del df_test['x']
df_test.head()

,image_id,class
0,Images/collar_design_labels/faad3490a16c7f3d4f...,collar_design_labels
1,Images/collar_design_labels/0b2b4254f35ce3a41a...,collar_design_labels
2,Images/collar_design_labels/7f2be608e06f804dd5...,collar_design_labels
3,Images/collar_design_labels/4b09d4dca80caac42e...,collar_design_labels
4,Images/collar_design_labels/de91f00a05e84d7239...,collar_design_labels


In [14]:
df_load = df_test[(df_test['class'] == cur_class)].copy()
df_load.reset_index(inplace=True)
del df_load['index']

print('{0}: {1}'.format(cur_class, len(df_load)))
df_load.head()

lapel_design_labels: 1343


,image_id,class
0,Images/lapel_design_labels/518bcadaa97418d29c7...,lapel_design_labels
1,Images/lapel_design_labels/156228a4743bfd86cef...,lapel_design_labels
2,Images/lapel_design_labels/393f4962b5c2fbf42cb...,lapel_design_labels
3,Images/lapel_design_labels/b5074c1b96f2402bab1...,lapel_design_labels
4,Images/lapel_design_labels/5ce450fa7e3f1890cb0...,lapel_design_labels


In [15]:
'./rank/{0}'.format(df_load['image_id'][0])

'./rank/Images/lapel_design_labels/518bcadaa97418d29c78a2f6dcff0209.jpg'

In [16]:
n = len(df_load)
X_test = np.zeros((n, width, width, 3), dtype=np.uint8)

for i in tqdm(range(n)):
    X_test[i] = cv2.resize(cv2.imread('./z_rank/{0}'.format(df_load['image_id'][i])), (width, width))


100%|██████████| 1343/1343 [00:07<00:00, 172.59it/s]


In [17]:
model.load_weights('./models/{}.best0420_InceptionV3_alldata.h5'.format(prefix_cls))

In [18]:
test_np = model.predict(X_test, batch_size=256)

In [19]:
test_np.shape

(1343, 5)

In [20]:
result = []

for i, row in df_load.iterrows():
    tmp_list = test_np[i]
    tmp_result = ''
    for tmp_ret in tmp_list:
        tmp_result += '{:.4f};'.format(tmp_ret)
        
    result.append(tmp_result[:-1])

df_load['result'] = result
df_load.head()

,image_id,class,result
0,Images/lapel_design_labels/518bcadaa97418d29c7...,lapel_design_labels,0.0000;0.0000;1.0000;0.0000;0.0000
1,Images/lapel_design_labels/156228a4743bfd86cef...,lapel_design_labels,0.9519;0.0069;0.0351;0.0050;0.0012
2,Images/lapel_design_labels/393f4962b5c2fbf42cb...,lapel_design_labels,0.7170;0.1644;0.0150;0.0811;0.0226
3,Images/lapel_design_labels/b5074c1b96f2402bab1...,lapel_design_labels,0.0000;0.0000;0.0001;0.0000;0.9999
4,Images/lapel_design_labels/5ce450fa7e3f1890cb0...,lapel_design_labels,0.9975;0.0007;0.0011;0.0003;0.0004


In [21]:
df_load.to_csv('./result/{}_0421alldata.csv'.format(prefix_cls), header=None, index=False)

In [22]:
df_load = pd.read_csv('./result/{}_0421alldata.csv'.format(prefix_cls), header=None)

In [23]:
len(df_load)

1343